# CNN-LSTM with Medically Realistic Augmentation

Self-contained notebook for Kaggle: loads the clean Alzheimer dataset, applies conservative augmentation (±15° rotations, no flips), trains an enhanced CNN+LSTM, and reports metrics/saves the model.

> Set `DATA_ROOT` below for Kaggle (`/kaggle/input/alzheimer-clean-dataset/Alzheimer_Clean_Dataset`) or local.

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
)

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Reshape, LSTM
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# GPU config (safe defaults for Kaggle/local)
print("GPU Configuration:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"  Using {len(gpus)} GPU(s)")
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("  GPU memory growth enabled")
    except RuntimeError:
        print("  GPU already initialized; memory growth skipped")
else:
    print("  No GPU detected; using CPU")

print(f"TensorFlow version: {tf.__version__}")


2025-12-10 12:13:23.776095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765368803.964751      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765368804.014031      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

GPU Configuration:
  Using 1 GPU(s)
  GPU memory growth enabled
TensorFlow version: 2.18.0


In [2]:
# DATA PATH: set for Kaggle or local
# Kaggle: DATA_ROOT = "/kaggle/input/alzheimer-clean-dataset/Alzheimer_Clean_Dataset"
# Local:  DATA_ROOT = r"d:\\ABDULLAH UNI\\Semester 7\\DL\\Classification-of-Alzheimer-s-disease-MRI-data-using-Deep-Learning\\Alzheimer_Clean_Dataset"
DATA_ROOT = "/kaggle/input/alzheimer-clean-dataset/Alzheimer_Clean_Dataset"

# Label mapping (binary collapse)
LABEL_MAP = {
    "NonDemented": "NonDemented",
    "VeryMildDemented": "Demented",
    "MildDemented": "Demented",
    "ModerateDemented": "Demented"
}

assert os.path.exists(DATA_ROOT), f"DATA_ROOT not found: {DATA_ROOT}"


In [3]:
# Helpers: load dataset metadata with label mapping

def create_dataframe(split_dir):
    rows = []
    for class_name in os.listdir(split_dir):
        class_path = os.path.join(split_dir, class_name)
        if os.path.isdir(class_path):
            mapped_label = LABEL_MAP.get(class_name, class_name)
            for f in os.listdir(class_path):
                if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                    rows.append({
                        'filename': os.path.join(class_path, f),
                        'label': mapped_label
                    })
    return pd.DataFrame(rows)

train_df = create_dataframe(os.path.join(DATA_ROOT, 'train'))
test_df = create_dataframe(os.path.join(DATA_ROOT, 'test'))

print("Train size:", len(train_df))
print(train_df['label'].value_counts())
print("\nTest size:", len(test_df))
print(test_df['label'].value_counts())

Train size: 5120
label
Demented       2560
NonDemented    2560
Name: count, dtype: int64

Test size: 1280
label
Demented       640
NonDemented    640
Name: count, dtype: int64


In [4]:
# Realistic augmentation generators (from improved-methodology)

def create_realistic_augmentation_generator():
    return ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        horizontal_flip=False,
        vertical_flip=False,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range=[0.9, 1.1],
        fill_mode='nearest'
    )

def create_no_augmentation_generator():
    return ImageDataGenerator(rescale=1./255)

def create_data_generators(input_size=(128, 128), batch_size=8):
    classes = ['NonDemented', 'Demented']
    train_datagen = create_realistic_augmentation_generator()
    val_datagen = create_no_augmentation_generator()
    train_gen = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='filename',
        y_col='label',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='binary',
        classes=classes,
        color_mode='rgb',
        shuffle=True,
        seed=SEED
    )
    val_gen = val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='filename',
        y_col='label',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='binary',
        classes=classes,
        color_mode='rgb',
        shuffle=False
    )
    return train_gen, val_gen

train_gen, val_gen = create_data_generators(batch_size=8)
print("Generators ready.")

Found 5120 validated image filenames belonging to 2 classes.
Found 1280 validated image filenames belonging to 2 classes.
Generators ready.


In [5]:
# Model: CNN-LSTM (simpler + stronger regularization for overfit control)

from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.regularizers import l2


def build_cnn_lstm_model(input_size=(128, 128)):
    model = models.Sequential(name="CNN_LSTM_Realistic_Aug_Aligned")
    model.add(Input(shape=(*input_size, 3)))
    # Conv Block 1
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    # Conv Block 2
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    # Conv Block 3
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.35))
    # Conv Block 4
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    # Reshape for LSTM bridge
    model.add(Reshape((8*8, 256)))
    model.add(TimeDistributed(Dense(128, activation='relu', kernel_regularizer=l2(1e-4))))
    model.add(BatchNormalization())
    model.add(Dropout(0.35))
    # Single LSTM (smaller)
    model.add(LSTM(64, dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(1e-4)))
    model.add(Dropout(0.45))
    # Classifier head (compact): 128 -> 64 -> 2
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(1e-3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(1e-3)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    # Use standard sparse categorical crossentropy (label smoothing unavailable in this TF version)
    model.compile(optimizer=Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_cnn_lstm_model()
model.summary()


I0000 00:00:1765368835.359907      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "CNN_LSTM_Realistic_Aug_Aligned"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 685,026 (2.61 MB)

 Trainable params: 683,106 (2.61 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [6]:
print("="*80)
print("STEP 6: TRAINING WITH DYNAMIC REALISTIC AUGMENTATION")
print("="*80)

# IMPROVED Callbacks for better training
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',  # Monitor accuracy instead of loss
        patience=20,  # More patience to allow convergence
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Moderate LR reduction (less aggressive)
        patience=10,  # Wait longer before reducing
        min_lr=1e-6,  # Higher minimum LR
        verbose=1,
        mode='min'
    )
]

print("\n Training Configuration:")
print("    Epochs: 150 (with early stopping)")
print("    Batch size: 64 (increased for GPU efficiency)")
print("    Optimizer: Adam (lr=0.001) - Original working config")
print("    Augmentation: DYNAMIC (new transforms each epoch)")
print("    Expected accuracy: ~85%+ (as achieved before)")
print("\n Starting training...\n")

# Train
start_time = time.time()

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=150,  # More epochs
    callbacks=callbacks,
    verbose=1
)

training_time = (time.time() - start_time) / 60

print(f"\n Training complete!")
print(f"   Time: {training_time:.2f} minutes")


STEP 6: TRAINING WITH DYNAMIC REALISTIC AUGMENTATION

 Training Configuration:
    Epochs: 150 (with early stopping)
    Batch size: 64 (increased for GPU efficiency)
    Optimizer: Adam (lr=0.001) - Original working config
    Augmentation: DYNAMIC (new transforms each epoch)
    Expected accuracy: ~85%+ (as achieved before)

 Starting training...



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/150


E0000 00:00:1765368847.454199      48 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/CNN_LSTM_Realistic_Aug_Aligned_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1765368848.862625     108 cuda_dnn.cc:529] Loaded cuDNN version 90300


640/640 ━━━━━━━━━━━━━━━━━━━━ 142s 199ms/step - accuracy: 0.4932 - loss: 1.5350 - val_accuracy: 0.5000 - val_loss: 0.9623 - learning_rate: 3.0000e-04
Epoch 2/150
640/640 ━━━━━━━━━━━━━━━━━━━━ 125s 195ms/step - accuracy: 0.5207 - loss: 1.2314 - val_accuracy: 0.5148 - val_loss: 0.9434 - learning_rate: 3.0000e-04
Epoch 3/150
640/640 ━━━━━━━━━━━━━━━━━━━━ 124s 193ms/step - accuracy: 0.5131 - loss: 1.0788 - val_accuracy: 0.5000 - val_loss: 0.9444 - learning_rate: 3.0000e-04
Epoch 4/150
640/640 ━━━━━━━━━━━━━━━━━━━━ 125s 195ms/step - accuracy: 0.5068 - loss: 1.0269 - val_accuracy: 0.4313 - val_loss: 0.9352 - learning_rate: 3.0000e-04
Epoch 5/150
640/640 ━━━━━━━━━━━━━━━━━━━━ 125s 195ms/step - accuracy: 0.5034 - loss: 0.9796 - val_accuracy: 0.5047 - val_loss: 0.9267 - learning_rate: 3.0000e-04
Epoch 6/150
640/640 ━━━━━━━━━━━━━━━━━━━━ 125s 195ms/step - accuracy: 0.4983 - loss: 0.9538 - val_accuracy: 0.5000 - val_loss: 0.9221 - learning_rate: 3.0000e-04
Epoch 7/150
640/640 ━━━━━━━━━━━━━━━━━━━━ 127s 

In [7]:
# Evaluation helpers

def calculate_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred) * 100,
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1_score': f1_score(y_true, y_pred, zero_division=0),
        'specificity': tn / (tn + fp) if (tn + fp) else 0,
        'cm': cm
    }
    return metrics

# Predict and report
val_gen.reset()
pred_probs = model.predict(val_gen, verbose=1)
pred_classes = np.argmax(pred_probs, axis=1)
true_classes = val_gen.classes
metrics = calculate_metrics(true_classes, pred_classes)

print("Metrics (%):", {k: round(v, 2) if k=='accuracy' else v for k, v in metrics.items() if k!='cm'})
print("Confusion matrix:\n", metrics['cm'])
print("\nClassification report:\n", classification_report(true_classes, pred_classes, target_names=['NonDemented','Demented']))


160/160 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step
Metrics (%): {'accuracy': 92.34, 'precision': 0.9385113268608414, 'recall': 0.90625, 'f1_score': 0.9220985691573926, 'specificity': 0.940625}
Confusion matrix:
 [[602  38]
 [ 60 580]]

Classification report:
               precision    recall  f1-score   support

 NonDemented       0.91      0.94      0.92       640
    Demented       0.94      0.91      0.92       640

    accuracy                           0.92      1280
   macro avg       0.92      0.92      0.92      1280
weighted avg       0.92      0.92      0.92      1280



In [8]:
# IEEE-style visualizations (high-res, Times New Roman)
from sklearn.metrics import roc_curve, auc

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.size': 10,
    'figure.dpi': 300,
    'axes.labelsize': 10,
    'axes.titlesize': 11,
})

os.makedirs('outputs', exist_ok=True)

# ROC curve
fpr, tpr, _ = roc_curve(true_classes, pred_probs[:, 1])
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(4, 4))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.3f}', color='navy')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (CNN-LSTM)')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('outputs/IEEE_CNN_LSTM_ROC.png', dpi=300, bbox_inches='tight')
plt.close()

# Confusion matrix (IEEE layout)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(4, 3.5))
sns.heatmap(cm_norm, annot=cm, fmt='d', cmap='Blues',
            xticklabels=['NonDemented', 'Demented'],
            yticklabels=['NonDemented', 'Demented'],
            cbar_kws={'label': 'Proportion'})
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (CNN-LSTM)')
plt.tight_layout()
plt.savefig('outputs/IEEE_CNN_LSTM_Confusion.png', dpi=300, bbox_inches='tight')
plt.close()

# Training curves (IEEE layout)
plt.figure(figsize=(6, 2.5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('outputs/IEEE_CNN_LSTM_Training.png', dpi=300, bbox_inches='tight')
plt.close()

print("Saved IEEE-ready figures to outputs/ (ROC, confusion, training curves).")

findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'serif' not found because none of the following families were found: Times New Roman
findfont: 

NameError: name 'cm' is not defined

In [ ]:
# Save model and plots

os.makedirs('outputs', exist_ok=True)
model.save('outputs/CNN_LSTM_Realistic_Aug.h5')

# Training curves
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title('Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('outputs/CNN_LSTM_training_history.png', dpi=150)
plt.show()

# Confusion matrix plot
cm = metrics['cm']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NonD','Dem'], yticklabels=['NonD','Dem'])
plt.title('Confusion Matrix')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.savefig('outputs/CNN_LSTM_confusion_matrix.png', dpi=150)
plt.show()

print("Saved model and plots to outputs/.")
